In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook  
import capo.redcal as om
from capo.metrics import DataContainer as dc

## TODO:
* Replace and upgrade build_reds
* Add remove_degen
* Add tests for remove_degen
* Add end-to-end 2d array tests

In [2]:
def build_reds_hex(hexNum, sep=14.7):
    antpos, i = {}, 0
    for row in range(hexNum-1,-(hexNum),-1):
        for col in range(2*hexNum-abs(row)-1):
            xPos = ((-(2*hexNum-abs(row))+2)/2.0 + col)*sep;
            yPos = row*sep*3**.5/2;
            antpos[i] = np.array([xPos, yPos, 0])
            i += 1
    return om.get_reds(antpos), antpos


In [11]:
reds, antpos = build_reds_hex(3)
rc = om.RedundantCalibrator(reds, antpos)
#gains,d = om.sim_red_data(reds, ['xx','yy','xy','yx'], gain_scatter=.01)
gains,true_vis,d = om.sim_red_data(reds, ['xx'], gain_scatter=.01)
w = dict([(k,1.) for k in d.keys()])
sol0 = dict([(k,np.ones_like(v)) for k,v in gains.items()])
sol0.update(rc.compute_ubls(d,sol0))
meta, sol = rc.lincal(d, sol0)

In [12]:
def remove_degen(sol, degen_sol=None):
    """This function removes the omnical degeneracies. If degen_sol is not specific, the amplitude terms are set to 1.0 and the phase terms are set to 0.0. Only full resolves degeneracies properly in the single polarization case or when Stokes V is artificially minimized by """
    ants = [key for key in sol.keys() if len(key)==2]
    positions = [rc.antpos[ant[0]] for ant in ants]
    bl_pairs = [key for key in sol.keys() if len(key)==3]
    bl_vecs = np.array([rc.antpos[bl_pair[0]] - rc.antpos[bl_pair[1]] for bl_pair in bl_pairs])
    
    #Extract gain and model visibiltiy solutions
    gainSols = np.array([sol[ant] for ant in ants])
    visSols = np.array([sol[bl_pair] for bl_pair in bl_pairs])
    if degen_sol is None: degenGains = np.ones_like(gainSols)
    else: degenGains = np.array([degen_sol[ant] for ant in ants])
    
    #Amplitude renormalization
    ampRenorm = (np.mean(np.abs(degenGains), axis=0) / np.mean(np.abs(gainSols), axis=0))
    newGainSols = gainSols * ampRenorm
    newVisSols = visSols / ampRenorm

    #Fix phase degeneracies
    Rgains = np.array([np.append(pos,1.0) for pos in positions])
    Mgains = np.linalg.pinv(Rgains.T.dot(Rgains)).dot(Rgains.T)
    Rvis = np.hstack((-bl_vecs, np.zeros((len(bl_pairs),1))))
    degenRemoved = np.einsum('ij,jkl',Mgains, np.angle(newGainSols)-np.angle(degenGains))
    newGainSols *= np.exp(-1.0j * np.einsum('ij,jkl',Rgains,degenRemoved)) 
    newVisSols *= np.exp(-1.0j * np.einsum('ij,jkl',Rvis,degenRemoved)) 
    
    #Create new solutions dictionary
    newSol = {}
    for ant,gainSol in zip(ants,newGainSols): newSol[ant] = gainSol
    for bl_pair,visSol in zip(bl_pairs,newVisSols): newSol[bl_pair] = visSol
    return newSol
    
remove_degen(sol0);

In [13]:
print sol0.keys(), '\n'
print [red[0] for red in reds], '\n'
print d.keys()
d.values()[0].shape

[(0, 9, 'xx'), (0, 14, 'xx'), (18, 'x'), (0, 17, 'xx'), (13, 'x'), (14, 'x'), (0, 15, 'xx'), (1, 7, 'xx'), (0, 5, 'xx'), (2, 'x'), (0, 12, 'xx'), (0, 18, 'xx'), (0, 8, 'xx'), (4, 3, 'xx'), (4, 12, 'xx'), (8, 'x'), (3, 13, 'xx'), (0, 16, 'xx'), (3, 'x'), (6, 7, 'xx'), (1, 4, 'xx'), (2, 12, 'xx'), (0, 3, 'xx'), (11, 7, 'xx'), (12, 'x'), (7, 'x'), (6, 3, 'xx'), (17, 'x'), (0, 6, 'xx'), (2, 6, 'xx'), (2, 15, 'xx'), (0, 4, 'xx'), (1, 13, 'xx'), (1, 'x'), (0, 7, 'xx'), (2, 16, 'xx'), (10, 'x'), (5, 'x'), (6, 'x'), (1, 12, 'xx'), (16, 'x'), (11, 'x'), (0, 13, 'xx'), (2, 0, 'xx'), (15, 'x'), (0, 'x'), (1, 0, 'xx'), (0, 10, 'xx'), (3, 11, 'xx'), (0, 11, 'xx'), (4, 'x'), (1, 3, 'xx'), (2, 7, 'xx'), (2, 3, 'xx'), (1, 16, 'xx'), (9, 'x')] 

[(3, 11), (0, 11), (0, 6), (0, 15), (0, 10), (0, 18), (0, 5), (0, 14), (0, 9), (0, 17), (3, 13), (0, 4), (0, 13), (2, 6), (2, 15), (0, 8), (0, 16), (0, 3), (0, 12), (1, 4), (1, 13), (4, 3), (4, 12), (1, 0), (0, 7), (1, 16), (1, 3), (1, 12), (2, 0), (1, 7), (2, 

(10, 10)

In [14]:
ants = [key for key in sol0.keys() if len(key)==2]
print ants

[(18, 'x'), (13, 'x'), (14, 'x'), (2, 'x'), (8, 'x'), (3, 'x'), (12, 'x'), (7, 'x'), (17, 'x'), (1, 'x'), (10, 'x'), (5, 'x'), (6, 'x'), (16, 'x'), (11, 'x'), (15, 'x'), (0, 'x'), (4, 'x'), (9, 'x')]
